In [ ]:
!pip install vllm

In [ ]:
!pip install -q vllm transformers accelerate

from vllm import LLM, SamplingParams

llm = LLM(model="gpt2")

sampling_params = SamplingParams(temperature=0.7, max_tokens=50)

prompts = ["The capital of India is"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(output.outputs[0].text)


In [2]:

#AssertionError: tensor model parallel group is not initialized


# GPT2Model is not intended to be used standalone in most cases.

# It requires internal setup from vLLM's distributed engine (like initializing tensor/pipeline parallel groups).

## chat gpt says

#This handles all the setup internally, including distributed backend, tensor parallelism, caching, model loading, and memory optimization.


from vllm import LLM, SamplingParams

llm = LLM(model="gpt2")


INFO 04-23 10:01:49 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 04-23 10:01:49 [config.py:689] This model supports multiple tasks: {'score', 'reward', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 04-23 10:01:49 [arg_utils.py:1731] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-23 10:01:49 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

INFO 04-23 10:02:02 [weight_utils.py:281] Time spent downloading weights for gpt2: 8.112076 seconds
INFO 04-23 10:02:03 [weight_utils.py:315] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-23 10:02:03 [loader.py:458] Loading weights took 0.51 seconds
INFO 04-23 10:02:03 [model_runner.py:1146] Model loading took 0.2378 GiB and 9.256457 seconds
INFO 04-23 10:02:05 [worker.py:267] Memory profiling takes 1.17 seconds
INFO 04-23 10:02:05 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 04-23 10:02:05 [worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 04-23 10:02:06 [executor_base.py:112] # cuda blocks: 22815, # CPU blocks: 7281
INFO 04-23 10:02:06 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 356.48x
INFO 04-23 10:02:10 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 04-23 10:02:43 [model_runner.py:1598] Graph capturing finished in 33 secs, took 0.14 GiB
INFO 04-23 10:02:43 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 39.75 seconds


In [17]:
from transformers import GPT2Config
from vllm.config import VllmConfig, ModelConfig, CacheConfig
from vllm.model_executor.models.gpt2 import GPT2Model


from transformers import GPT2Config



hf_config = GPT2Config()
hf_config.vocab_size = 10682
hf_config.n_layer = 16
hf_config.n_positions = 2048
hf_config.n_embd = 512
hf_config.n_head = 8

model_config = ModelConfig(
    model="gpt2",
    task="generate",
    tokenizer="gpt2",
    tokenizer_mode="auto",
    trust_remote_code=False,
    dtype="auto",
    seed=42,
    hf_config_path=None,
)
model_config.hf_config = hf_config

cache_config = CacheConfig(
    block_size=8,
    gpu_memory_utilization=0.1,
    swap_space=4,
    cache_dtype="auto"
)

vllm_config = VllmConfig(
    model_config=model_config,
    cache_config=cache_config,
)

gpt_model = GPT2Model(vllm_config=vllm_config)


INFO 04-23 10:14:54 [config.py:2832] Downcasting torch.float32 to torch.float16.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM config is not set.
WARNING 04-23 10:14:54 [config.py:3934] Current vLLM co

In [13]:
gpt_model.state_dict().keys()

odict_keys(['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight', 'h.3.ln_1.bias', 'h.3.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.3.attn.c_proj.weight', 'h.3.attn.c_proj.bias', 'h.3.ln_2.weight', 'h.3.ln_2.bia

In [6]:
!wget https://huggingface.co/Dubverse/MahaTTS/resolve/main/maha_tts/pretrained_models/Smolie-en/t2s_best.pt


--2025-04-23 10:04:33--  https://huggingface.co/Dubverse/MahaTTS/resolve/main/maha_tts/pretrained_models/Smolie-en/t2s_best.pt
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.23, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e4/c1/e4c10f18697efa05f94cbc9159cc5d02d7e878b2a51adfde09eab8362e513d8b/6be1b489366ebbd35e55404be875804d380b0430587319f67e592da7ba1b5240?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27t2s_best.pt%3B+filename%3D%22t2s_best.pt%22%3B&Expires=1745406273&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTQwNjI3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2U0L2MxL2U0YzEwZjE4Njk3ZWZhMDVmOTRjYmM5MTU5Y2M1ZDAyZDdlODc4YjJhNTFhZGZkZTA5ZWFiODM2MmU1MTNkOGIvNmJlMWI0ODkzNjZlYmJkMzVlNTU0MDRiZTg3NTgwNGQzODBiMDQzMDU4NzMxOWY2N2U1OTJkYTdiYTFiNTI

In [22]:
import torch

checkpoint = torch.load("t2s_best.pt")

print(checkpoint["state_dict"].keys())
new_state_dict = {k.replace("gpt.", ""): v for k, v in checkpoint["state_dict"].items()}

odict_keys(['gpt.h.0.ln_1.weight', 'gpt.h.0.ln_1.bias', 'gpt.h.0.attn.c_attn.weight', 'gpt.h.0.attn.c_attn.bias', 'gpt.h.0.attn.c_proj.weight', 'gpt.h.0.attn.c_proj.bias', 'gpt.h.0.ln_2.weight', 'gpt.h.0.ln_2.bias', 'gpt.h.0.mlp.c_fc.weight', 'gpt.h.0.mlp.c_fc.bias', 'gpt.h.0.mlp.c_proj.weight', 'gpt.h.0.mlp.c_proj.bias', 'gpt.h.1.ln_1.weight', 'gpt.h.1.ln_1.bias', 'gpt.h.1.attn.c_attn.weight', 'gpt.h.1.attn.c_attn.bias', 'gpt.h.1.attn.c_proj.weight', 'gpt.h.1.attn.c_proj.bias', 'gpt.h.1.ln_2.weight', 'gpt.h.1.ln_2.bias', 'gpt.h.1.mlp.c_fc.weight', 'gpt.h.1.mlp.c_fc.bias', 'gpt.h.1.mlp.c_proj.weight', 'gpt.h.1.mlp.c_proj.bias', 'gpt.h.2.ln_1.weight', 'gpt.h.2.ln_1.bias', 'gpt.h.2.attn.c_attn.weight', 'gpt.h.2.attn.c_attn.bias', 'gpt.h.2.attn.c_proj.weight', 'gpt.h.2.attn.c_proj.bias', 'gpt.h.2.ln_2.weight', 'gpt.h.2.ln_2.bias', 'gpt.h.2.mlp.c_fc.weight', 'gpt.h.2.mlp.c_fc.bias', 'gpt.h.2.mlp.c_proj.weight', 'gpt.h.2.mlp.c_proj.bias', 'gpt.h.3.ln_1.weight', 'gpt.h.3.ln_1.bias', 'gpt.h.3

In [24]:
dissimilar_strings = set(list(gpt_model.state_dict().keys())) ^ set(list(new_state_dict.keys()))


In [25]:
dissimilar_strings

{'GST.reference_encoder.0.bias',
 'GST.reference_encoder.0.weight',
 'GST.reference_encoder.1.bias',
 'GST.reference_encoder.1.weight',
 'GST.reference_encoder.2.norm.bias',
 'GST.reference_encoder.2.norm.weight',
 'GST.reference_encoder.2.proj_out.bias',
 'GST.reference_encoder.2.proj_out.weight',
 'GST.reference_encoder.2.qkv.bias',
 'GST.reference_encoder.2.qkv.weight',
 'GST.reference_encoder.2.relative_pos_embeddings.relative_attention_bias.weight',
 'GST.reference_encoder.3.norm.bias',
 'GST.reference_encoder.3.norm.weight',
 'GST.reference_encoder.3.proj_out.bias',
 'GST.reference_encoder.3.proj_out.weight',
 'GST.reference_encoder.3.qkv.bias',
 'GST.reference_encoder.3.qkv.weight',
 'GST.reference_encoder.3.relative_pos_embeddings.relative_attention_bias.weight',
 'GST.reference_encoder.4.norm.bias',
 'GST.reference_encoder.4.norm.weight',
 'GST.reference_encoder.4.proj_out.bias',
 'GST.reference_encoder.4.proj_out.weight',
 'GST.reference_encoder.4.qkv.bias',
 'GST.reference_e

In [26]:
gpt1 = gpt_model

In [27]:
from collections import OrderedDict

model_dict = gpt_model.state_dict()
pretrained_dict = {k: v for k, v in new_state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
model_dict.update(pretrained_dict)
gpt_model.load_state_dict(model_dict)

<All keys matched successfully>

In [29]:
len(gpt_model.state_dict().keys())

196